# Elasticsearch Low-Level Python API
## Index operations
### Assumes Star Wars indices are loaded

In [3]:
#import sys, requests, time, json
from elasticsearch import Elasticsearch

## Connect to Cluster

In [5]:
es_conn1 = Elasticsearch([{'host': '10.0.2.15', 'port': 9200}])
print(es_conn1.cluster.health())

{'cluster_name': 'parrot_elk', 'status': 'yellow', 'timed_out': False, 'number_of_nodes': 1, 'number_of_data_nodes': 1, 'active_primary_shards': 9, 'active_shards': 9, 'relocating_shards': 0, 'initializing_shards': 0, 'unassigned_shards': 8, 'delayed_unassigned_shards': 0, 'number_of_pending_tasks': 0, 'number_of_in_flight_fetch': 0, 'task_max_waiting_in_queue_millis': 0, 'active_shards_percent_as_number': 52.94117647058824}


## Index Operations

In [6]:
# close an index, useful to just stop R/W ops
# but does not delete it
# close the sw_people
es_conn1.indices.close('sw_people')

{'acknowledged': True,
 'shards_acknowledged': True,
 'indices': {'sw_people': {'closed': True}}}

In [7]:
# open the closed index sw_people
es_conn1.indices.open('sw_people')

{'acknowledged': True, 'shards_acknowledged': True}

In [8]:
# create an index named delete_me
es_conn1.indices.create('delete_me')

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'delete_me'}

In [9]:
# check to make sure it exists 
# it should
print('index exists: ', es_conn1.indices.exists('delete_me'))

index exists:  True


In [11]:
# delete index delete_me
es_conn1.indices.delete('delete_me')

{'acknowledged': True}

In [12]:
# check to make sure it exists 
# it should
print('index exists: ', es_conn1.indices.exists('delete_me'))

index exists:  False


In [13]:
# flush indexes
# this will flush sw_people
es_conn1.indices.exists('sw_people')
#  this flushes them all
es_conn1.indices.exists('_all')

True

In [14]:
# force merge
# this will reduce the number of segments in the shards
# this will lock the index until the force merge completes
# SO BEWARE
es_conn1.indices.forcemerge('sw_people')

{'_shards': {'total': 2, 'successful': 1, 'failed': 0}}

In [15]:
# refresh Explicitly refresh one or more index
# making all operations performed since the last refresh available for search
# not entirley sure what that gives us
es_conn1.indices.refresh('sw_people')

{'_shards': {'total': 2, 'successful': 1, 'failed': 0}}

In [19]:
# returns store info for shards
print('shard store info',es_conn1.indices.shard_stores('sw_people'))

shard store info {'indices': {'sw_people': {'shards': {'0': {'stores': [{'93tVwMVdRyKf8eqHq2bF4w': {'name': 'parrot', 'ephemeral_id': 'Hxq4TXdCRHG-Ot9xF3iuQQ', 'transport_address': '10.0.2.15:9300', 'attributes': {}}, 'allocation_id': 'd8Gk463YTP2r9ieEkBI6nw', 'allocation': 'primary'}]}}}}}


In [18]:
# low level info on segments
print('index segement info: ', es_conn1.indices.segments('sw_people'))
# this will show all the segments for all the indices
#print('all segment info', es_conn1.indices.segments('_all'))
print('\nPython type: ', type(es_conn1.indices.segments('_all')))

index segement info:  {'_shards': {'total': 2, 'successful': 1, 'failed': 0}, 'indices': {'sw_people': {'shards': {'0': [{'routing': {'state': 'STARTED', 'primary': True, 'node': '93tVwMVdRyKf8eqHq2bF4w'}, 'num_committed_segments': 2, 'num_search_segments': 2, 'segments': {'_k': {'generation': 20, 'num_docs': 67, 'deleted_docs': 0, 'size_in_bytes': 37989, 'memory_in_bytes': 10147, 'committed': True, 'search': True, 'version': '8.2.0', 'compound': True, 'attributes': {'Lucene50StoredFieldsFormat.mode': 'BEST_SPEED'}}, '_l': {'generation': 21, 'num_docs': 20, 'deleted_docs': 0, 'size_in_bytes': 24413, 'memory_in_bytes': 9522, 'committed': True, 'search': True, 'version': '8.2.0', 'compound': False, 'attributes': {'Lucene50StoredFieldsFormat.mode': 'BEST_SPEED'}}}}]}}}}

Python type:  <class 'dict'>


In [31]:
# shrink sounds useful but not going to do it
# allows you to shrink an existing index into a new index with fewer primary shards
# The number of primary shards in the target index must be a factor of the shards in the source index.
# an index with 8 primary shards can be shrunk into 4, 2 or 1 primary shards
# an index with 15 primary shards can be shrunk into 5, 3 or 1.
# Before shrinking, a (primary or replica) copy of every shard in the index must be present on the same node.
# and this isn't fully functional
#es_conn1.indices.shrink(index='sw_people', target='sw_people_shrink')
# 1- create target index
# 2- make it read-only

### need to get the shrink command straightened out

In [32]:
# stats shows ops on an index
# a lot of info
index_stats = es_conn1.indices.stats('sw_people')
print(index_stats)
print('\nPython type: ', type(index_stats))

{'_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_all': {'primaries': {'docs': {'count': 87, 'deleted': 0}, 'store': {'size_in_bytes': 62820}, 'indexing': {'index_total': 0, 'index_time_in_millis': 0, 'index_current': 0, 'index_failed': 0, 'delete_total': 0, 'delete_time_in_millis': 0, 'delete_current': 0, 'noop_update_total': 0, 'is_throttled': False, 'throttle_time_in_millis': 0}, 'get': {'total': 1, 'time_in_millis': 5, 'exists_total': 1, 'exists_time_in_millis': 5, 'missing_total': 0, 'missing_time_in_millis': 0, 'current': 0}, 'search': {'open_contexts': 0, 'query_total': 6, 'query_time_in_millis': 6, 'query_current': 0, 'fetch_total': 4, 'fetch_time_in_millis': 0, 'fetch_current': 0, 'scroll_total': 0, 'scroll_time_in_millis': 0, 'scroll_current': 0, 'suggest_total': 0, 'suggest_time_in_millis': 0, 'suggest_current': 0}, 'merges': {'current': 0, 'current_docs': 0, 'current_size_in_bytes': 0, 'total': 0, 'total_time_in_millis': 0, 'total_docs': 0, 'total_size_in_bytes': 0,